In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv("C:/Users/bektursun/PycharmProjects/pythonProject/train.csv")

In [2]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) #shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255

In [3]:
X_train[:,0].shape

(784,)

In [4]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2
  
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    Z_exp = np.exp(Z - np.max(Z, axis = 0))
    return Z_exp / np.sum(Z_exp, axis=0)
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1) 
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = 2 * (A2 - one_hot_Y)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, 1)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, 1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * np.reshape(db1, (10, 1))    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * np.reshape(db2, (10, 1))    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

Iteration:  0
0.10075609756097562
Iteration:  10
0.2658048780487805
Iteration:  20
0.4337560975609756
Iteration:  30
0.5498536585365854
Iteration:  40
0.6271463414634146
Iteration:  50
0.6839024390243903
Iteration:  60
0.7229512195121951
Iteration:  70
0.7506585365853659
Iteration:  80
0.7698292682926829
Iteration:  90
0.785
Iteration:  100
0.7971463414634147
Iteration:  110
0.8073414634146342
Iteration:  120
0.8144390243902438
Iteration:  130
0.8215121951219512
Iteration:  140
0.8266341463414634
Iteration:  150
0.8316829268292683
Iteration:  160
0.8370487804878048
Iteration:  170
0.8405609756097561
Iteration:  180
0.8439512195121951
Iteration:  190
0.8472926829268292
Iteration:  200
0.8497560975609756
Iteration:  210
0.8525121951219512
Iteration:  220
0.8549268292682927
Iteration:  230
0.8574878048780488
Iteration:  240
0.8597073170731707
Iteration:  250
0.8618292682926829
Iteration:  260
0.8633170731707317
Iteration:  270
0.8648780487804878
Iteration:  280
0.8664878048780488
Iteratio

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()